# Protein NPT simulation

In [25]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import MDAnalysis as mda
import nglview as nv

In [26]:
# process protein
fixer=PDBFixer(pdbid='5xg0')
fixer.removeChains(chainIds=["B","C"])
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(keepWater=False)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
fixer.addSolvent(padding=10*angstroms,positiveIon='Na+',negativeIon='Cl-')
PDBFile.writeFile(fixer.topology,fixer.positions,open('5xg0_process.pdb','w'))

In [27]:
# view processed protein using nglview
view=nv.show_mdanalysis(mda.Universe('5xg0_process.pdb'))
view.add_licorice('water',opacity=0.35)
view.add_spacefill('ion')
view.center('all')
view

NGLWidget()

In [28]:
# simulation settings
pdbfile='5xg0_process.pdb' # processd protein pdbfile

nonbondedmethod=PME   # method to use for nonbonded interactions. NoCutoff | CutoffNonPeriodic | CutoffPeriodic | Ewald | PME | LJPME 
nonbondedcutoff=1*nanometer  # cutoff distance for nonboned interactions
constraints= HBonds   # which bonds or angles to be implemented with constraints. None, HBonds, AllBonds, or HAngles
removecmmotion=True    # if remove center of mass motion
rigidwater = True      # If true, water molecules will be fully rigid
ewaldErrorTolerance = 0.0005  
constraintTolerance = 0.000001     
hydrogenMass = 1.5*amu   # set the mass of hydrogen

dt = 0.002*picoseconds   # step size
temperature = 300*kelvin   # simulating temperature
friction = 1.0/picosecond  # the friction coefficient which couples the system to the heat bath
pressure = 1.0*atmospheres  # pressure
barostatInterval = 25       #  the frequency at which Monte Carlo pressure changes should be attempted

equilibrationSteps=10000    # equil simulation steps
steps=20000                # simulation steps
 
dcd=True                    # if or not save traj in dcd format
dcdstep=1000                # step to save dcd 
chk=True                    # if or not save checkpoint file
chkstep=1000                # step to save checkpoint file
data=stdout               # output data in stdout or file. stdout | 'log.txt
datastep=100               # step to output data

dcdReporter = DCDReporter('trajectory.dcd', dcdstep)
dataReporter = StateDataReporter(data,datastep, totalSteps=steps,
    step=True, speed=True, progress=True, potentialEnergy=True, temperature=True,density=True,volume=True,separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', chkstep)

platform=Platform.getPlatformByName("CUDA")  #  CUDA | CPU |OPENGL
properties={"DeviceIndex":"0","Precision":"single"}  #  single double

In [29]:
pdb=PDBFile(pdbfile)
forcefield=ForceField('amber14-all.xml','amber14/tip3p.xml')
system=forcefield.createSystem(pdb.topology,nonbondedMethod=nonbondedmethod,
                                nonbondedCutoff=nonbondedcutoff,
                                constraints=constraints,
                                removeCMMotion=removecmmotion,
                                rigidWater=rigidwater,
                                ewaldErrorTolerance=ewaldErrorTolerance, 
                                hydrogenMass=hydrogenMass
                                )
# 另一种控温
# system.addForce(AndersenThermostat(temperature,friction))
# integrator = VerletIntegrator(0.002*picoseconds)
system.addForce(MonteCarloBarostat(pressure,temperature,barostatInterval))

5

In [30]:
integrator=LangevinIntegrator(temperature,friction,dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation=Simulation(pdb.topology,system,integrator,platform,proterties)
simulation.context.setPositions(pdb.positions)

In [31]:
# Minimizing
simulation.minimizeEnergy()

In [32]:
# Equil
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

In [33]:
# Simulating
if dcd:
    simulation.reporters.append(dcdReporter)
else:
    pass
simulation.reporters.append(dataReporter)
if chk:
    simulation.reporters.append(checkpointReporter)
else:
    pass
simulation.currentStep = 0
simulation.step(steps)
positions=simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology,positions,open("lastframe.pdb","w"))

#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Box Volume (nm^3)"	"Density (g/mL)"	"Speed (ns/day)"
0.5%	100	-560294.605358453	300.941876441936	440.0772236257678	1.0141894503545472	0
1.0%	200	-560252.105358453	298.6282782323613	440.0772236257678	1.0141894503545472	239
1.5%	300	-560908.2044523465	299.59427773279504	439.9112745120185	1.0145720362308623	245
2.0%	400	-560928.9804884265	299.8325381211868	438.62819090281516	1.0175398818391541	245
2.5%	500	-561091.7288735718	300.0802252146156	439.4552709810704	1.015624813297092	246
3.0%	600	-561026.9942469536	299.32038496225096	439.07673799952556	1.0165003948422673	246
3.5%	700	-561382.5170352892	302.6527606554545	439.63787090802606	1.0152029819922967	245
4.0%	800	-559764.0170352892	299.12904449473086	439.63787090802606	1.0152029819922967	245
4.5%	900	-560294.7051030449	303.04644099312367	439.12021745212616	1.0163997461383831	244
5.0%	1000	-560317.3193678362	302.9022913341364	440.76342684028924	1.0126105079591763	205
5

In [34]:
# view the last frame pdb file
view=nv.show_mdanalysis(mda.Universe('lastframe.pdb'))
view.add_licorice('water',opacity=0.35)
view.add_spacefill('ion')
view.center('all')
view

NGLWidget()

In [35]:
traj=nv.show_mdanalysis(mda.Universe('lastframe.pdb','trajectory.dcd'))
# traj.add_licorice('water',opacity=0.35)
# traj.add_spacefill('ion')
traj

/home/casea/CASEADATA/software/Anaconda/envs/openmm/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


NGLWidget(max_frame=19)